In [2]:
#from pydub import AudioSegment
import numpy as np
import matplotlib as plt
from decimal import Decimal
import math

#audio = AudioSegment.from_mp3("samples/440Hz_44100Hz_16bit_05sec.mp3")
#samples = np.array(audio.get_array_of_samples())
# audio = AudioSegment.from_mp3("samples/Bas - Tribe with J.Cole.mp3")
# samples1 = np.array(audio.get_array_of_samples())
# samples,len(samples),samples1,len(samples1)


#trimmed = np.where(samples != 0)[0]
# trimmed = trimmed[:100]
#samples.shape

$\omega$ = frequency variable  
$F[n]$ = sample $n$

# Discrete Fourier Transform
inputs: Sequence of complex pairs in the time domain
outputs: Sequence of complex pairs in the frequency domain

### Equations: 
#### DFT
$X(k)=\sum_{t=0}^{n-1}x(t)e^{-2\pi itk/n}$

#### Euler's formula:  
$e^{ix} = \cos x+i\sin x$

#### DFT+Euler's:  
$e^{-2\pi itk/n} = \cos(-2\pi tk/n) + i\sin(-2\pi tk/n)$  
$(a+bi)(c+di) = ac+adi+bi-bd$  
$ac+adi+bi-bd = ac-bd+(ad+bc)i$  
$e^{-2\pi itk/n} = \cos(2\pi tk/n) - i\sin(2\pi tk/n)$  

#### DFT-eval
Let $R(x(t)) =$ the real component of $x(t)$  
Let $I(x(t)) =$ the complex componet of $x(t)$. $I(x(t)) = 0$ for all our data.  

$X(k)=\sum_{t=0}^{n-1}[R(x(t))+I(x(t))][\cos(2\pi tk/n) - i\sin(2\pi tk/n)]$  
$X(k)=\sum_{t=0}^{n-1}x(t)[\cos(2\pi tk/n) - i\sin(2\pi tk/n)]$  
$X(k)=\sum_{t=0}^{n-1}x(t)\cos(2\pi tk/n) - ix(t)\sin(2\pi tk/n)]$  


In [15]:
def fft(x):
    """
    Compute the FFT of the input sequence x using the Cooley-Tukey algorithm.

    Parameters:
    - x: Input sequence (array-like).

    Returns:
    - X: FFT of the input sequence.
    """
    N = len(x)
    if N <= 1:
        return x
    else:
        # Divide the input sequence into even and odd indices
        even = fft(x[::2])
        odd = fft(x[1::2])
        
        # Compute twiddle factors
        twiddles = np.exp(-2j * np.pi * np.arange(N) / N)
        
        # Combine the FFTs of even and odd indices using twiddle factors
        X = np.concatenate([even + twiddles[:N//2] * odd,
                            even + twiddles[N//2:] * odd])
        return X

def DFT(x):
    """
    Perform discrete fourier transform over input array x
    x is assumed to be real and of shape (n,1)
    
    Args
    ----
    x: Array to perform DFT over
    
    Returns
    -------
    out: Complex result of DFT, shape (n,2)
    """
    n = len(x)
    if n > 50_000:
        print("You're trying to do >50,000^2 operations. Cancelling for you, this is probably a mistake")
        return 0
    out = []
    def kstep(k:int)->complex:
        """
        Calculate the k step sum for DFT
        
        Args
        ----
        int:k - the step to calculate
        
        Return
        complex:step - the complex number result
        """
        # r,i = 0,0j
        
        # print(2*np.pi*k/n)
        # print(k/n)
        angle = np.arange(0,n)*2*np.pi*k/n
        r = np.sum(x*np.cos(angle))
        i = np.sum(-x*np.sin(angle))
#         for t in range(0,n):
#         {
            
#             # angle = (2*np.pi*t*k/n)
#             # print(x[t])
            
#             # r += x[t]*np.cos(angle)
#             # i += -x[t]*np.sin(angle)
#             # if r != 0 or i != 0:
#             #     print(t,r,i)
#         }
        return complex(r,i)

    for k in range(n):
        out.append(kstep(k))
    return out

def slicer(x,size=500,slices=None):
    """
    Given an array (x,_) split the array into 'slices' sub-arrays of length <= size
    If slices isn't provided, slices = ceil(len(x)/size). 
    
    If size isn't a factor of len(x), size will pad the last slice with 0's
    
    If slices and size are provided, slices will take precidence.
    
    For a given slices, size will become 1+len(x)//slices for all except 
    the last m slices, which will have size len(x)//slices. This is to keep the
    slice sizes consistent with each other. 
    **In practice this will be done by duplicating the last values of all m slices.**
    maybe want to just fill 0? need to experiment, this seems insignificant
    
    
    Args
    ----
    x: Array to slice
    size: Max size of each slice
    slices: Number of slices to slice 
    
    Return
    ------
    a: Array of shape (slices,size)
    """
    n = len(x)
    a = []
    
    if n < size:
        print("Size too large for x")
        return a
    
    if not slices:
        #slices = int(np.ceil(n/size))
        slices = int(n // size)
        # print(slices)
        # a = np.zeros(shape=(slices,size)) we don't need to specify the length of each input into a
        # just append a new slice to a of len size
    # if slices:
    
    if n % slices == 0:
        # slices are a factor of n
        size = int(n/slices)

        for i in range(0,n,size):
            a.append(x[i:i + size]) # appends slices of x to a each of len size
    else:
        # remainder with slices
        size = int(np.ceil(n/slices))
        n_p = n%slices # number of slices with size+1
        # a = np.zeros((slices,size))
        remainder = np.zeros(n_p)

        for i in range(0,n,size):
            a.append(x[i:i + size])

        a += remainder

        # TODO: finish slicing logic. size and slice are calculations finished
        # TODO: validate slicing edge cases

    return a
        
# # DFT(trimmed)    
# slicer(trimmed,size=50_000),len(trimmed)/50_000

In [24]:
lst1 = [1,2,3,4,5,6,7,8,9,10] # size and slices given and slices is factor of len(x)
a = slicer(lst1,2,5)
a
lst2 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] # slices not given and size not a factor of len(x)
b = slicer(lst2,2)
print(f'a:{a}\nb:\n{b}')
# TODO: implement and test size not given and slices not factor of len(x) for slicer

a:[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]]
b:
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]
 [10. 11. 12.]
 [13. 14. 15.]]
